In [2]:
!pip install psycopg2

In [3]:
import pandas as pd

df = pd.read_csv("C:/Users/carlf/OneDrive/Bureau/NoSQL_SQL/bank_full.csv")

In [4]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default_', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [5]:
import psycopg2
# Connect to PostgreSQL database
conn = psycopg2.connect(dbname="new_db", user="postgres", password="#Skylanders6/", host="localhost",  port="5432")

In [6]:
# Create a cursor object
cur = conn.cursor()

# Define the table schema
table_name = "bank_full"
columns = [
    "age INTEGER",
    "job VARCHAR(100)",
    "marital VARCHAR(20)",
    "education VARCHAR(100)",
    "default_ VARCHAR(10)",
    "balance NUMERIC",
    "housing VARCHAR(10)",
    "loan VARCHAR(10)",
    "contact VARCHAR(100)",
    "day INTEGER",
    "month VARCHAR(20)",
    "duration INTEGER",
    "campaign INTEGER",
    "pdays INTEGER",
    "previous INTEGER",
    "poutcome VARCHAR(100)",
    "y VARCHAR(10)"
]

# Construct the CREATE TABLE query
create_table_query = (
    "CREATE TABLE IF NOT EXISTS " + table_name + " (" +
    ", ".join(columns) +
    ")"
)

# Execute the CREATE TABLE query
cur.execute(create_table_query)

# Commit the transaction
conn.commit()

# Close cursor and connection
cur.close()
conn.close()

In [7]:
# 1: What is the average age of the customers?
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(dbname="new_db", user="postgres", password="#Skylanders6/", host="localhost", port="5432")

# Create a cursor object
cur = conn.cursor()

# Execute the query to calculate the average age
cur.execute("SELECT AVG(age) FROM bank_full")

# Fetch the result
average_age = cur.fetchone()[0]

print("Average age of the customers:", average_age)

# Close cursor and connection
cur.close()
conn.close()

Average age of the customers: 40.9362102143283714


In [10]:
#2: How many customers have defaulted on loans?
conn = psycopg2.connect(dbname="new_db", user="postgres", password="#Skylanders6/", host="localhost", port="5432")
c = conn.cursor()

c.execute("""SELECT COUNT(*)
    FROM bank_full
    WHERE default_ = 'yes';
""")
          
default_loans = c.fetchone()[0]    
print(f"Le nombre de client à avoir un défaut de paiement est de : {default_loans}")

Le nombre de client à avoir un défaut de paiement est de : 815


In [12]:
# 3: What is the maximum balance in the account?
c.execute("SELECT MAX(balance) FROM bank_full")

max_balance = c.fetchone()[0]

print("Solde maximum dans le compte :", max_balance)

Solde maximum dans le compte : 102127


In [14]:
# 4: How many customers have a housing loan?
c.execute("SELECT COUNT(*) FROM bank_full WHERE housing = 'yes'")

num_customers_with_housing_loan = c.fetchone()[0]

print("Nombre de clients avec un prêt immobilier :", num_customers_with_housing_loan)

Nombre de clients avec un prêt immobilier : 25130


In [15]:
# 5: What is the education level of the youngest customer?
c.execute("SELECT education FROM bank_full ORDER BY age LIMIT 1")

education_youngest_customer = c.fetchone()[0]

print("Niveau d'éducation du client le plus jeune :", education_youngest_customer)

Niveau d'éducation du client le plus jeune : primary


In [16]:
# 6: What is the marital status of the oldest customer?
c.execute("SELECT marital FROM bank_full ORDER BY age DESC LIMIT 1")

marital_status_oldest_customer = c.fetchone()[0]

print("Statut matrimonial du client le plus âgé :", marital_status_oldest_customer)

Statut matrimonial du client le plus âgé : divorced


In [17]:
# 7: What is the distribution of the months in which customers were last contacted?
c.execute("SELECT month, COUNT(*) FROM bank_full GROUP BY month ORDER BY month")

month_distribution = c.fetchall()

print("Distribution des mois de dernier contact des clients :")
for month, count in month_distribution:
    print(month, ":", count)

Distribution des mois de dernier contact des clients :
apr : 2932
aug : 6247
dec : 214
feb : 2649
jan : 1403
jul : 6895
jun : 5341
mar : 477
may : 13766
nov : 3970
oct : 738
sep : 579


In [18]:
# 8: What is the average duration of contact for customers who have a personal loan?
c.execute("SELECT ROUND(AVG(duration), 2) FROM bank_full WHERE loan = 'yes'")

average_duration = c.fetchone()[0]

print("Durée moyenne de contact pour les clients avec un prêt personnel :", average_duration)

Durée moyenne de contact pour les clients avec un prêt personnel : 250.85


In [21]:
# 9: How many customers were previously contacted, and what was the average duration of their last contact?
c.execute("SELECT COUNT(*) FROM bank_full WHERE pdays != -1")
num_customers_contacted = c.fetchone()[0]

c.execute("SELECT ROUND(AVG(duration), 2) FROM bank_full WHERE pdays != -1")
average_duration_last_contact = c.fetchone()[0]

print("Nombre de clients précédemment contactés :", num_customers_contacted)
print("Durée moyenne du dernier contact des clients précédemment contactés :", average_duration_last_contact)

Nombre de clients précédemment contactés : 8257
Durée moyenne du dernier contact des clients précédemment contactés : 260.12


In [22]:
# 10: What is the distribution of job types among customers?
c.execute("SELECT job, COUNT(*) FROM bank_full GROUP BY job ORDER BY COUNT(*) DESC")

job_distribution = c.fetchall()

print("Distribution des types d'emploi parmi les clients :")
for row in job_distribution:
    print(row[0], ":", row[1])

Distribution des types d'emploi parmi les clients :
blue-collar : 9732
management : 9458
technician : 7597
admin. : 5171
services : 4154
retired : 2264
self-employed : 1579
entrepreneur : 1487
unemployed : 1303
housemaid : 1240
student : 938
unknown : 288


In [26]:
# 11: How many customers have been contacted more than once in the current campaign?
 
c.execute("""
        SELECT COUNT(*) FROM (
            SELECT age, job, marital, education, COUNT(*) 
            FROM bank_full 
            GROUP BY age, job, marital, education 
            HAVING COUNT(*) > 1
        ) AS subquery
    """)

num_customers_contacted_multiple_times = c.fetchone()[0]

print("Nombre de clients contactés plus d'une fois au cours de la campagne :", num_customers_contacted_multiple_times)


Nombre de clients contactés plus d'une fois au cours de la campagne : 2639


In [28]:
# 12: What is the success rate of previous marketing campaigns (poutcome) among customers
c.execute("SELECT COUNT(*) FROM bank_full")
total_customers = c.fetchone()[0]

c.execute("SELECT COUNT(*) FROM bank_full WHERE poutcome = 'success'")
successful_customers = c.fetchone()[0]

success_rate = (successful_customers / total_customers) * 100

print("Taux de réussite des campagnes marketing précédentes parmi les clients :", success_rate, "%")

Taux de réussite des campagnes marketing précédentes parmi les clients : 3.34210700935613 %
